## Ramblings: Naive Bayes, Algoritmos Generativos, Batch Feed y trabajando con texto

### Sobre Algoritmos Generativos vs. Discriminativos

> Caveat: Intuición de Ng, Andrew. 2013. CS299: Machine Learning. Department of Computer Science. Stanford University


Si tenemos dos tipos de animales, Elefantes ($y=1$) y Perros ($y=0$), buscamos predecir a una nueva observación en base a una serie de atributos $x\in\mathbf{X}$. Ante este problema, existen dos escuelas de pensamiento:

- __Algoritmos generativos__: Generamos un resumen o representación sobre cuáles son las principales características de un Elefante y Perro, en función a la comunalidad de atributos __para cada clase__. De esta manera estamos observando la probabilidad de clase __en función a la probabilidad conjunta de los atributos__. Con nuestros resumenes de cada clase, evaluamos si una observación presenta similitudes en algunos de los dos grupos y asignamos la clase en base al grupo con mayor similitud.

- __Algoritmos discrimminativos__: Buscamos generar una recta divisoria (llamada frontera de decisión) que busque separar la observaciones Elefantes y Perro. Posteriormente para clasificar una nueva observación, evaluamos en que lado de la recta se sitúa. 

#### ¿Y qué es mejor?

> Caveat: La intuición proviene de Jordan y Ng (2001), Shalev-Shwartz y Ben-David (2014)

Resulta que la principal diferencia entre un modelo generativo y uno discriminativo radica en la cantidad de casos:

- Un modelo generativo tiene una tasa de error asintótico más alto (condicional a la cantidad de observaciones en nuestro conjunto de entrenamiento).
- En los modelos generativos existe el paso intermedio donde evaluamos la probabilidad conjunta de los datos y la clase $\textsf{Pr}(\mathbf{X}, y)$. El problema de este paso es que puede capturar ruido antes de evaluar la estrategia de maximización a posteriori.
- La máxima sugiere implementar modelos discriminativos por sobre generativos, dado que evitamos este error de forma innecesaria.
- Un punto a favor de los generativos es que cuando la cantidad de datos de entrenamiento es baja, el modelo generativo alcanza la cota asintótica del error mucho más rápido que un modelo discriminativo.


### Métricas en problemas de clasificación

- __Precision__: Razón entre predicciones positivas correctas en el total de las predicciones positivas totales.
- __Recall__: Razón entre predicciones positivas correctas sobre el total de predicciones correctamente identificadas.


Existe un trade off que depende mucho del problema de clasificación al cual nos enfrentamos. Podemos estar en situaciones donde deseamos maximizar la precisión (evitar que una observación sea clasificada incorrectamente) y  no tomar tanto en cuenta el recall (podemos tolerar un grado menor de observaciones con falsos negativos)

Aún así, el óptimo es encontrar precision y recall altos.

### Batch Feed

* ¿Cómo realizar la ingesta de una cantidad substancial de datos?

In [1]:
!ls dump/

a_tribe_called_quest_scrape.csv     michael_jackson_scrape.csv
anthrax_scrape.csv                  mobb_deep_scrape.csv
black_star_scrape.csv               modest_mouse_scrape.csv
bob_dylan_scrape.csv                mos_def_scrape.csv
britney_spears_scrape.csv           necrophagist_scrape.csv
bruce_springsteen_scrape.csv        nickelback_scrape.csv
cannibal_corpse_scrape.csv          nicki_minaj_scrape.csv
carly_rae_jepsen_scrape.csv         nwa_scrape.csv
de_la_soul_scrape.csv               oasis_scrape.csv
deicide_scrape.csv                  opeth_scrape.csv
dr._dre_scrape.csv                  pink_floyd_scrape.csv
dua_lipa_scrape.csv                 public_enemy_scrape.csv
eminem_scrape.csv                   queen_scrape.csv
faith_no_more_scrape.csv            radiohead_scrape.csv
ghostface_killah_scrape.csv         raekwon_scrape.csv
gorgoroth_scrape.csv                rage_against_the_machine_scrape.csv
immortal_scrape.csv                 red_hot_chili_peppers_scrape.csv
incubus

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob, os

In [3]:
# generamos una lista con todos los csv existentes
file_list = glob.glob(os.getcwd() + '/dump/*.csv')
# lista vacía

append_csv = []
# para cada csv
for f in file_list:
    # abrir en pandas y concatenar
    append_csv.append(pd.read_csv(f, index_col=None, header=0).drop(columns='Unnamed: 0'))
# concatenar la lista de dataframes


df_lyrics = pd.concat(append_csv)
df_lyrics.columns = ['Artist', 'Genre', 'Song', 'Lyrics']

In [ ]:
# windows amigos - not tested though

a = [name for name in os.listdir("./dump/") if name.endswith(".csv")]
append_csv = []
# para cada csv
for f in file_list:
    # abrir en pandas y concatenar
    append_csv.append(pd.read_csv(f, index_col=None, header=0).drop(columns='Unnamed: 0'))
# concatenar la lista de dataframes
df_lyrics = pd.concat(append_csv)
df_lyrics.columns = ['Artist', 'Genre', 'Song', 'Lyrics']


In [5]:
df_lyrics.shape

(9489, 4)

### Trabajo con texto en `sklearn`

* Si tenemos una serie $N$ de documentos con $d$ frases y $w$ palabras, un algoritmo clásico de ML no podrá trabajar de forma bruta con la secuencia de valores alfanumérica. Razones:
    - No existe una unidad vectorial clara.
    - Los documentos puede tener un tamaño variante en frases y palabras.
    
* __Solución__: Nuestro objetivo va a ser transformar una serie de documentos en una matriz (dispersa). Pasos:
    - __Tokenizar__: Separaremos cada palabra existente en todos los documentos (también llamado corpus) y le asignaremos un identificador entero. Algunas decisiones de tokenización: Espacios en blanco, signos de puntuación.
    - __Contar__: la frecuencia de cada token __en cada documento__.
    - __Ponderar y normalizar__ aquellos tokens con una menor o mayor ocurrencia.
    
* Estructura de datos: 
    - cada palabra (o token) se puede considerar como un atributo específico.
    - cada documento se compone de distintas ocurrencias de tokens.
    
Este proceso se conoce como _Bag of Words_ $\leadsto$ Cada documento se describe como una combinación de ocurrencia de palabras, asumiendo __ignorabilidad completa__ a la posición relativa de la palabra en la oración.    
    
    
#### CountVectorizer

Implementa un conteo discreto de cada palabra en el _Bag-of-Words_. Existen otras variantes como `FeatureHasher`, `TfidfVectorizer` que buscan agilizar el proceso y ponderar por la ocurrencia general en el corpus.